In [80]:
import xarray as xr
import ocf_blosc2
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import dask.array as da
from dask.diagnostics import ProgressBar
import random
from tqdm import tqdm


In [81]:
nwp_data = xr.open_dataset("../../../mnt/disks/gcp_data/nwp/ecmwf/UK_v2.zarr")
nwp_data

<xarray.Dataset> Size: 1TB
Dimensions:    (variable: 14, init_time: 5049, step: 49, latitude: 241,
                longitude: 301)
Coordinates:
  * init_time  (init_time) datetime64[ns] 40kB 2017-01-01 ... 2023-12-31T12:0...
  * latitude   (latitude) float64 2kB 60.0 59.95 59.9 59.85 ... 48.1 48.05 48.0
  * longitude  (longitude) float64 2kB -12.0 -11.95 -11.9 ... 2.9 2.95 3.0
  * step       (step) timedelta64[ns] 392B 00:00:00 01:00:00 ... 2 days 00:00:00
  * variable   (variable) <U5 280B 't2m' 'dswrf' 'dlwrf' ... 'v10' 'u100' 'v100'
Data variables:
    ECMWF_UK   (variable, init_time, step, latitude, longitude) float32 1TB ...

In [82]:
meta_data = pd.read_csv("data_files/metadata.csv")
meta_data.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [83]:
pv_data = xr.open_dataset("data_files/pv.netcdf", engine='h5netcdf')
pv_data

<xarray.Dataset> Size: 2GB
Dimensions:   (datetime: 387254)
Coordinates:
  * datetime  (datetime) datetime64[ns] 3MB 2018-01-01T06:05:00 ... 2021-10-2...
Data variables: (12/1311)
    10003     (datetime) float32 2MB ...
    10004     (datetime) float32 2MB ...
    10005     (datetime) float32 2MB ...
    10048     (datetime) float32 2MB ...
    10049     (datetime) float32 2MB ...
    10063     (datetime) float32 2MB ...
    ...        ...
    9870      (datetime) float32 2MB ...
    9871      (datetime) float32 2MB ...
    9902      (datetime) float32 2MB ...
    9903      (datetime) float32 2MB ...
    9960      (datetime) float32 2MB ...
    9989      (datetime) float32 2MB ...

In [84]:
skip_ss_ids = ['8440', '16718', '8715', '17073', '9108', '9172', '10167', '10205', '10207', '10278', '26778', '26819', '10437', '10466', '26915', '10547', '26939', '26971', '10685', '10689', '2638', '2661', '2754', '2777', '2783', '2786', '2793', '2812', '2829', '2830', '2867', '2883', '2904', '2923', '2947', '2976', '2989', '2999', '3003', '3086', '3118', '3123', '3125', '3264', '3266', '3271', '3313', '3334', '3470', '3502', '11769', '11828', '11962', '3772', '11983', '3866', '3869', '4056', '4067', '4116', '4117', '4124', '4323', '4420', '20857', '4754', '13387', '13415', '5755', '5861', '5990', '6026', '6038', '6054', '14455', '6383', '6430', '6440', '6478', '6488', '6541', '6548', '6560', '14786', '6630', '6804', '6849', '6868', '6870', '6878', '6901', '6971', '7055', '7111', '7124', '7132', '7143', '7154', '7155', '7156', '7158', '7201', '7237', '7268', '7289', '7294', '7311', '7329', '7339', '7379', '7392', '7479', '7638', '7695', '7772', '15967', '7890', '16215', '7830']

In [85]:
# select hourly pv data all of it, except the skip ids. 
# select the rest and plot graphs of years with generation.

print(len(skip_ss_ids))
hourly_pv_data = pv_data.sel(datetime=pv_data['datetime'].dt.minute == 0)

valid_ss_ids_data = [var for var in hourly_pv_data.data_vars if var not in skip_ss_ids]
pv_sites_id = np.random.choice(valid_ss_ids_data, 500, replace=False)
filtered_hourly_pv_data = hourly_pv_data[pv_sites_id]
filtered_hourly_pv_data

119


<xarray.Dataset> Size: 65MB
Dimensions:   (datetime: 32193)
Coordinates:
  * datetime  (datetime) datetime64[ns] 258kB 2018-01-01T07:00:00 ... 2021-10...
Data variables: (12/500)
    27012     (datetime) float32 129kB ...
    12451     (datetime) float32 129kB ...
    15846     (datetime) float32 129kB ...
    8648      (datetime) float32 129kB ...
    10425     (datetime) float32 129kB ...
    7088      (datetime) float32 129kB ...
    ...        ...
    6966      (datetime) float32 129kB ...
    11438     (datetime) float32 129kB ...
    26986     (datetime) float32 129kB ...
    26792     (datetime) float32 129kB ...
    9153      (datetime) float32 129kB ...
    6482      (datetime) float32 129kB ...

In [86]:
def get_36_hour_range(start_datetime, hours=36):
    end_datetime = start_datetime + pd.Timedelta(hours=hours - 1, minutes=59)
    return start_datetime, end_datetime

def select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours):
    selected_datetimes = []
    available_datetimes = list(datetimes)

    for _ in range(num_selections):
        if not available_datetimes:
            break
        random_datetime = np.random.choice(available_datetimes)
        selected_datetimes.append(random_datetime)
        
        # Remove datetimes within the min_gap_hours range
        available_datetimes = [dt for dt in available_datetimes if dt > random_datetime + pd.Timedelta(hours=min_gap_hours)]

    return selected_datetimes

In [87]:
datetimes = pd.to_datetime(filtered_hourly_pv_data['datetime'].values)
data_dict = {'ss_id': [], 'pv_datetime': [], 'generation' : [], 'horizon':[]}

batch_size = 36
num_selections = 5000
min_gap_hours = 36


for ss_id in pv_sites_id:
    selected_datetimes = select_non_overlapping_datetimes(datetimes, num_selections, min_gap_hours)
    
    for start_datetime in selected_datetimes:
        start, end = get_36_hour_range(start_datetime, hours=batch_size)
        selected_data = hourly_pv_data.sel(datetime=slice(start, end))

        if len(selected_data['datetime']) < batch_size or selected_data[ss_id].isnull().any():
            continue
        
        hour_counter = 1  # Initialize hour_counter for each new batch
        batch_data = {'ss_id': [], 'pv_datetime': [], 'generation': [], 'horizon': []}
        
        for dt, power in zip(selected_data['datetime'].values, selected_data[ss_id].values):
            batch_data['ss_id'].append(int(ss_id))
            batch_data['pv_datetime'].append(dt)
            batch_data['generation'].append(power)
            batch_data['horizon'].append(hour_counter)
            hour_counter += 1
        
        if hour_counter - 1 == batch_size:
            for key in data_dict.keys():
                data_dict[key].extend(batch_data[key])
        else:
            pass

In [88]:
pv_df = pd.DataFrame(data_dict)
print(pv_df.shape)
pv_df = pv_df.dropna(subset={'generation'})

pv_df
print(pv_df.shape)


(55296, 4)
(55296, 4)


In [89]:
# pv_df.to_csv("tr2.csv")

In [90]:
# pv_df = pd.read_csv("tr2.csv")
# pv_df

In [91]:
pv_sites_id = [int(id) for id in pv_sites_id]
pv_site_dict = {'ss_id':[], "lat":[], "long": [], 'tilt':[], 'orientation':[], 'kwp':[]}

for id in pv_sites_id:
    row = meta_data[meta_data['ss_id'] == id]
    if not row.empty:
        pv_site_dict['ss_id'].append(id)
        pv_site_dict['lat'].append(row['latitude_rounded'].values[0])
        pv_site_dict['long'].append(row['longitude_rounded'].values[0])
        pv_site_dict['tilt'].append(row['tilt'].values[0])
        pv_site_dict['orientation'].append(row['orientation'].values[0])
        pv_site_dict['kwp'].append(row['kwp'].values[0])
    else:
        print('row empty')
        

meta_site_df = pd.DataFrame.from_dict(pv_site_dict)
meta_site_df

,ss_id,lat,long,tilt,orientation,kwp
0,27012,51.59,-1.81,36.0,225.0,3.99
1,12451,57.34,-2.33,40.0,145.0,3.50
2,15846,54.32,-2.76,25.0,165.0,2.00
3,8648,50.37,-5.06,35.0,70.0,4.00
4,10425,53.62,-2.13,26.0,160.0,2.00
...,...,...,...,...,...,...
495,11438,54.53,-3.57,43.0,174.0,2.00
496,26986,51.80,-0.10,45.0,180.0,2.00
497,26792,53.47,-2.19,35.0,130.0,2.50
498,9153,51.23,-2.30,35.0,215.0,3.76


In [92]:
combined_df = pd.merge(pv_df, meta_site_df, on='ss_id', how='inner')
combined_df

,ss_id,pv_datetime,generation,horizon,lat,long,tilt,orientation,kwp
0,27012,2019-03-28 00:00:00,0.000000,1,51.59,-1.81,36.0,225.0,3.99
1,27012,2019-03-28 01:00:00,0.000000,2,51.59,-1.81,36.0,225.0,3.99
2,27012,2019-03-28 02:00:00,0.000000,3,51.59,-1.81,36.0,225.0,3.99
3,27012,2019-03-28 03:00:00,0.000000,4,51.59,-1.81,36.0,225.0,3.99
4,27012,2019-03-28 04:00:00,0.000000,5,51.59,-1.81,36.0,225.0,3.99
...,...,...,...,...,...,...,...,...,...
55291,6482,2021-09-30 13:00:00,263.284790,32,50.38,-4.03,30.0,150.0,3.84
55292,6482,2021-09-30 14:00:00,180.533997,33,50.38,-4.03,30.0,150.0,3.84
55293,6482,2021-09-30 15:00:00,146.733597,34,50.38,-4.03,30.0,150.0,3.84
55294,6482,2021-09-30 16:00:00,129.042007,35,50.38,-4.03,30.0,150.0,3.84


In [93]:
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])
combined_df['pv_date'] = combined_df['pv_datetime'].dt.date
combined_df['pv_hour'] = combined_df['pv_datetime'].dt.hour
combined_df

,ss_id,pv_datetime,generation,horizon,lat,long,tilt,orientation,kwp,pv_date,pv_hour
0,27012,2019-03-28 00:00:00,0.000000,1,51.59,-1.81,36.0,225.0,3.99,2019-03-28,0
1,27012,2019-03-28 01:00:00,0.000000,2,51.59,-1.81,36.0,225.0,3.99,2019-03-28,1
2,27012,2019-03-28 02:00:00,0.000000,3,51.59,-1.81,36.0,225.0,3.99,2019-03-28,2
3,27012,2019-03-28 03:00:00,0.000000,4,51.59,-1.81,36.0,225.0,3.99,2019-03-28,3
4,27012,2019-03-28 04:00:00,0.000000,5,51.59,-1.81,36.0,225.0,3.99,2019-03-28,4
...,...,...,...,...,...,...,...,...,...,...,...
55291,6482,2021-09-30 13:00:00,263.284790,32,50.38,-4.03,30.0,150.0,3.84,2021-09-30,13
55292,6482,2021-09-30 14:00:00,180.533997,33,50.38,-4.03,30.0,150.0,3.84,2021-09-30,14
55293,6482,2021-09-30 15:00:00,146.733597,34,50.38,-4.03,30.0,150.0,3.84,2021-09-30,15
55294,6482,2021-09-30 16:00:00,129.042007,35,50.38,-4.03,30.0,150.0,3.84,2021-09-30,16


In [94]:
combined_df.to_csv("result_data/combined_df.csv")

In [95]:
combined_df['pv_datetime'] = pd.to_datetime(combined_df['pv_datetime'])
combined_df

,ss_id,pv_datetime,generation,horizon,lat,long,tilt,orientation,kwp,pv_date,pv_hour
0,27012,2019-03-28 00:00:00,0.000000,1,51.59,-1.81,36.0,225.0,3.99,2019-03-28,0
1,27012,2019-03-28 01:00:00,0.000000,2,51.59,-1.81,36.0,225.0,3.99,2019-03-28,1
2,27012,2019-03-28 02:00:00,0.000000,3,51.59,-1.81,36.0,225.0,3.99,2019-03-28,2
3,27012,2019-03-28 03:00:00,0.000000,4,51.59,-1.81,36.0,225.0,3.99,2019-03-28,3
4,27012,2019-03-28 04:00:00,0.000000,5,51.59,-1.81,36.0,225.0,3.99,2019-03-28,4
...,...,...,...,...,...,...,...,...,...,...,...
55291,6482,2021-09-30 13:00:00,263.284790,32,50.38,-4.03,30.0,150.0,3.84,2021-09-30,13
55292,6482,2021-09-30 14:00:00,180.533997,33,50.38,-4.03,30.0,150.0,3.84,2021-09-30,14
55293,6482,2021-09-30 15:00:00,146.733597,34,50.38,-4.03,30.0,150.0,3.84,2021-09-30,15
55294,6482,2021-09-30 16:00:00,129.042007,35,50.38,-4.03,30.0,150.0,3.84,2021-09-30,16


In [101]:
results = []
batch_size = 36

In [102]:
counter = 0
for i in tqdm(range(0, len(combined_df), batch_size), desc="Processing batches"):
    batch = combined_df.iloc[i:i + batch_size]
    
    if len(batch) < batch_size:
        continue  # Skip incomplete batches

    # Get the initial pv_datetime and corresponding init_time in nwp dataset
    initial_time = batch.iloc[0]['pv_datetime']
    
    # Get the latitude and longitude from the first row in the batch
    lat = batch.iloc[0]['lat']
    lon = batch.iloc[0]['long']
    
    # Select the nearest latitude and longitude in the NWP dataset first
    nwp_sel = nwp_data.sel(latitude=lat, method="nearest").sel(longitude=lon, method="nearest")
    
    # Select the nearest init_time
    init_time_sel = nwp_sel.sel(init_time=initial_time, method="backfill")
    
    if init_time_sel.init_time.size == 0:
        continue  # No matching init_time found

    # matching_init_time = init_time_sel.init_time.values[0]

    # Retrieve the 36 steps of forecast data starting from the matched init_time
    data_sel = init_time_sel.sel(step=slice(pd.Timedelta(hours=0), pd.Timedelta(hours=35)))

    # Convert to DataFrame and pivot
    data_df = data_sel.to_dataframe().reset_index()
    pivot_df = data_df.pivot_table(index=['init_time', 'step'], columns='variable', values='ECMWF_UK').reset_index()

    if len(pivot_df) < batch_size:
        continue

    for j in range(batch_size):
        pivot_df.loc[j, 'ss_id'] = batch.iloc[j]['ss_id']
        pivot_df.loc[j, 'pv_datetime'] = batch.iloc[j]['pv_datetime']
        pivot_df.loc[j, 'generation'] = batch.iloc[j]['generation']
        pivot_df.loc[j, 'horizon'] = batch.iloc[j]['horizon']
        pivot_df.loc[j, 'lat'] = lat
        pivot_df.loc[j, 'long'] = lon
        pivot_df.loc[j, 'tilt'] = batch.iloc[j]['tilt']
        pivot_df.loc[j, 'orientation'] = batch.iloc[j]['orientation']
        pivot_df.loc[j, 'kwp'] = batch.iloc[j]['kwp']
        pivot_df.loc[j, 'pv_hour'] = batch.iloc[j]['pv_hour']

    # Append to the results list
    results.append(pivot_df)
    counter += 1


Processing batches: 100%|██████████| 1536/1536 [04:07<00:00,  6.20it/s]


In [103]:
print(len(results))
print(counter)

1536
1536


In [104]:
final_df = pd.concat(results, ignore_index=True)
final_df

variable,init_time,step,dlwrf,dswrf,duvrs,hcc,lcc,mcc,sde,sr,...,ss_id,pv_datetime,generation,horizon,lat,long,tilt,orientation,kwp,pv_hour
0,2019-03-28 00:00:00,0 days 00:00:00,0.000000e+00,0.0,0.0,0.431641,0.264862,0.000000,0.0,0.0,...,27012.0,2019-03-28 00:00:00,0.000000,1.0,51.59,-1.81,36.0,225.0,3.99,0.0
1,2019-03-28 00:00:00,0 days 01:00:00,9.701556e+05,0.0,0.0,0.278198,0.099609,0.000000,0.0,0.0,...,27012.0,2019-03-28 01:00:00,0.000000,2.0,51.59,-1.81,36.0,225.0,3.99,1.0
2,2019-03-28 00:00:00,0 days 02:00:00,1.928544e+06,0.0,0.0,0.210449,0.038666,0.000000,0.0,0.0,...,27012.0,2019-03-28 02:00:00,0.000000,3.0,51.59,-1.81,36.0,225.0,3.99,2.0
3,2019-03-28 00:00:00,0 days 03:00:00,2.887392e+06,0.0,0.0,0.345367,0.013062,0.001373,0.0,0.0,...,27012.0,2019-03-28 03:00:00,0.000000,4.0,51.59,-1.81,36.0,225.0,3.99,3.0
4,2019-03-28 00:00:00,0 days 04:00:00,3.837596e+06,0.0,0.0,0.171173,0.053040,0.053558,0.0,0.0,...,27012.0,2019-03-28 04:00:00,0.000000,5.0,51.59,-1.81,36.0,225.0,3.99,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55291,2021-09-29 12:00:00,1 days 07:00:00,3.765654e+07,9142437.0,1100624.5,0.963440,1.000000,0.891052,0.0,10934765.0,...,6482.0,2021-09-30 13:00:00,263.284790,32.0,50.38,-4.03,30.0,150.0,3.84,13.0
55292,2021-09-29 12:00:00,1 days 08:00:00,3.904446e+07,9142437.0,1100624.5,0.974854,1.000000,0.767334,0.0,10934765.0,...,6482.0,2021-09-30 14:00:00,180.533997,33.0,50.38,-4.03,30.0,150.0,3.84,14.0
55293,2021-09-29 12:00:00,1 days 09:00:00,4.043992e+07,9142437.0,1100624.5,0.981506,0.574188,0.143158,0.0,10934765.0,...,6482.0,2021-09-30 15:00:00,146.733597,34.0,50.38,-4.03,30.0,150.0,3.84,15.0
55294,2021-09-29 12:00:00,1 days 10:00:00,4.172285e+07,9142437.0,1100624.5,0.960266,0.721252,0.161621,0.0,10934765.0,...,6482.0,2021-09-30 16:00:00,129.042007,35.0,50.38,-4.03,30.0,150.0,3.84,16.0


In [105]:
final_df.columns

Index(['init_time', 'step', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc',
       'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100', 'ss_id',
       'pv_datetime', 'generation', 'horizon', 'lat', 'long', 'tilt',
       'orientation', 'kwp', 'pv_hour'],
      dtype='object', name='variable')

In [106]:
desired_order = ['ss_id', 'init_time', 'step', 'pv_datetime', 'pv_hour', 'horizon', 'generation', 'kwp', 'lat', 'long', 'tilt', 'orientation', 'dlwrf', 'dswrf', 'duvrs', 'hcc', 'lcc', 'mcc', 'sde', 'sr', 't2m', 'tcc', 'u10', 'u100', 'v10', 'v100']
final_df = final_df[desired_order]

In [107]:
final_df

variable,ss_id,init_time,step,pv_datetime,pv_hour,horizon,generation,kwp,lat,long,...,lcc,mcc,sde,sr,t2m,tcc,u10,u100,v10,v100
0,27012.0,2019-03-28 00:00:00,0 days 00:00:00,2019-03-28 00:00:00,0.0,1.0,0.000000,3.99,51.59,-1.81,...,0.264862,0.000000,0.0,0.0,277.958252,0.581116,1.584653,0.916919,-0.676881,-2.600140
1,27012.0,2019-03-28 00:00:00,0 days 01:00:00,2019-03-28 01:00:00,1.0,2.0,0.000000,3.99,51.59,-1.81,...,0.099609,0.000000,0.0,0.0,277.939209,0.350616,-0.527094,-1.395622,-2.033790,-3.317603
2,27012.0,2019-03-28 00:00:00,0 days 02:00:00,2019-03-28 02:00:00,2.0,3.0,0.000000,3.99,51.59,-1.81,...,0.038666,0.000000,0.0,0.0,277.731934,0.240021,-1.722570,-2.422046,-0.668341,-2.065826
3,27012.0,2019-03-28 00:00:00,0 days 03:00:00,2019-03-28 03:00:00,3.0,4.0,0.000000,3.99,51.59,-1.81,...,0.013062,0.001373,0.0,0.0,277.155762,0.353699,-1.373006,-2.166355,-1.160448,-1.858932
4,27012.0,2019-03-28 00:00:00,0 days 04:00:00,2019-03-28 04:00:00,4.0,5.0,0.000000,3.99,51.59,-1.81,...,0.053040,0.053558,0.0,0.0,275.155762,0.219604,-1.260833,-2.253342,-2.072457,-2.646996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55291,6482.0,2021-09-29 12:00:00,1 days 07:00:00,2021-09-30 13:00:00,13.0,32.0,263.284790,3.84,50.38,-4.03,...,1.000000,0.891052,0.0,10934765.0,288.531982,1.000007,6.516399,10.268157,6.878848,10.431475
55292,6482.0,2021-09-29 12:00:00,1 days 08:00:00,2021-09-30 14:00:00,14.0,33.0,180.533997,3.84,50.38,-4.03,...,1.000000,0.767334,0.0,10934765.0,288.644043,1.000007,7.194014,11.384347,6.527172,9.857162
55293,6482.0,2021-09-29 12:00:00,1 days 09:00:00,2021-09-30 15:00:00,15.0,34.0,146.733597,3.84,50.38,-4.03,...,0.574188,0.143158,0.0,10934765.0,288.809326,0.990262,6.851295,10.688770,4.804249,7.091730
55294,6482.0,2021-09-29 12:00:00,1 days 10:00:00,2021-09-30 16:00:00,16.0,35.0,129.042007,3.84,50.38,-4.03,...,0.721252,0.161621,0.0,10934765.0,288.439209,0.982205,6.030088,9.574234,3.938220,5.938288


In [108]:
final_df.to_csv("result_data/tft_data_36_final.csv")